Щоб запустити наступні ноутбуки, якщо ви ще цього не зробили, вам потрібно встановити ключ openai у файлі .env як `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Далі ми завантажимо Індекс Вбудов у Dataframe Pandas. Індекс Вбудов збережено у JSON-файлі під назвою `embedding_index_3m.json`. Індекс Вбудов містить вбудови для кожного з транскриптів YouTube до кінця жовтня 2023 року.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Далі ми створимо функцію під назвою `get_videos`, яка буде шукати в Індексі Вбудовувань за запитом. Функція поверне топ-5 відео, які найбільше схожі на запит. Функція працює наступним чином:

1. Спочатку створюється копія Індексу Вбудовувань.
2. Далі обчислюється Вбудовування для запиту за допомогою OpenAI Embedding API.
3. Потім у Індексі Вбудовувань створюється новий стовпець під назвою `similarity`. Стовпець `similarity` містить косинусну схожість між Вбудовуванням запиту та Вбудовуванням для кожного сегмента відео.
4. Далі Індекс Вбудовувань фільтрується за стовпцем `similarity`. Індекс Вбудовувань фільтрується так, щоб включати лише відео з косинусною схожістю більшою або рівною 0.75.
5. Нарешті, Індекс Вбудовувань сортується за стовпцем `similarity`, і повертаються топ-5 відео.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ця функція дуже проста, вона просто виводить результати пошукового запиту.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Спочатку індекс вбудов завантажується у Pandas Dataframe.
2. Далі користувачу пропонується ввести запит.
3. Потім викликається функція `get_videos` для пошуку в індексі вбудов за запитом.
4. Нарешті, викликається функція `display_results` для відображення результатів користувачу.
5. Потім користувачу знову пропонується ввести запит. Цей процес триває, доки користувач не введе `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.uk.png)

Вам буде запропоновано ввести запит. Введіть запит і натисніть Enter. Додаток поверне список відео, які відповідають запиту. Додаток також поверне посилання на місце у відео, де знаходиться відповідь на питання.

Ось кілька запитів для спроби:

- Що таке Azure Machine Learning?
- Як працюють згорткові нейронні мережі?
- Що таке нейронна мережа?
- Чи можна використовувати Jupyter Notebooks з Azure Machine Learning?
- Що таке ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ рідною мовою слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується звертатися до професійного людського перекладу. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникли внаслідок використання цього перекладу.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
